In [1]:
import vk
import pandas as pd
import random
import time
from sklearn.externals import joblib
from scipy.sparse import csr_matrix
from pathlib import Path
from sklearn.decomposition import TruncatedSVD

## Function to get movies info from VK

In [2]:
def get_data(id, n, min_num_of_movies=3):
    
    movies = pd.Series()
    index = 0
    
    file = Path("id")
    if file.is_file():
        id = joblib.load("id")
        movies = joblib.load("movies")
        index = movies.shape[0]
    last_id = 0
    count = 0
    
    while movies.size < n:
        
        f = vk.get_friends(str(id))
        print(f)

        friends = []
        for x in f:
            friends.append(str(x))

        for friend in friends:
            if movies.size >= n:
                break
            
            try:
                if count == 100:
                    print("pause")
                    time.sleep(5)
                    count = 0
                else:
                    count += 1
                    
                user_info = vk.get_users_info([friend])

                if ("movies" in user_info[friend].keys() 
                    and len(user_info[friend]["movies"].split(",")) >= min_num_of_movies
                    and not friend in movies.keys()
                    and len(friends) >= 5):
                    if index % 20 == 0:
                        joblib.dump(movies, filename="movies1", compress=9)
                        joblib.dump(friend, filename="id1", compress=9)
                    movies.set_value(index, user_info[friend]["movies"].split(","))
                    index += 1
                    print(friend)
            except:
                pass
        print("Index: " + str(index))
        print("Id: " + str(id))
        print("Last id: " + str(last_id))
        if movies.size < n:
            last_id = id
            while True: 
                ind = random.randint(0, len(friends)-1)
                if (len(vk.get_friends(friends[ind])) >= 5
                    and last_id != friends[ind]):
                    id = friends[ind]
                    break
            
    return movies

## Function to get unique movies

In [3]:
def get_unique_movies(data):
    unique_movies = pd.Series()
    index = 0
    for movies in data:
        for movie in movies:
            if not movie in unique_movies:
                unique_movies.set_value(index, movie)
                index += 1
    return unique_movies

## Function to construct a matrix

In [4]:
def construct_matrix(data, unique_movies):
    row = []
    column = []

    for user in data.keys():
        for movie in data[user]:
            row.append(user)
            column.append(unique_movies[unique_movies == movie].index[0])
    ones = [1 for x in range(len(row))]
    
    movies_matrix = csr_matrix((ones, (row, column)), \
                               shape=(len(data), len(unique_movies)))
    
    a = movies_matrix.toarray()
    
    bad_indexes = [x for x in range(len(a))]
    while len(bad_indexes) != 0:
        maxes = a.argmax(1)
        for index in bad_indexes:
            if (a[index][maxes[index]] == 1):
                del bad_indexes[bad_indexes.index(index)]
            else:
                a[index][maxes[index]] = 1
    
    movies_matrix = csr_matrix(a)
    
    return movies_matrix

# Main stuff

In [ ]:
file = Path("movies")
#if file.is_file():
#data = joblib.load("movies")
#else:
data = get_data(1, 100000)

[585393, 1528402, 2456454, 3894889, 4265053, 4504636, 4682791, 5381557, 6683264, 7523329, 7883598, 8918396, 8998418, 9096440, 9265411, 9630471, 10313202, 10313286, 10526671, 10827624, 11039705, 11405088, 11612849, 11720139, 12796318, 13166219, 13297719, 13324826, 13537597, 13882694, 14128193, 14286434, 14443565, 14679596, 14689526, 14923394, 15538621, 15796700, 16225037, 16230254, 16314467, 17197969, 17618306, 17724609, 18727671, 18780606, 19516462, 20280726, 20342794, 21318464, 21727395, 22684438, 22873234, 23442045, 25798340, 25926984, 26512291, 28201640, 28225111, 28644524, 28913134, 29325102, 29497169, 29725375, 30168544, 30477310, 30638541, 31269836, 32316705, 32497261, 34728040, 35413263, 35432630, 36726699, 36737888, 37011526, 37708348, 38454237, 38461299, 39157919, 39218007, 39586501, 39962367, 40279983, 41482851, 41526383, 41595840, 41767724, 41790715, 42186368, 42422768, 42734345, 43271569, 44308487, 44848435, 44917460, 50283596, 50819879, 51903744, 52394605, 53445352, 548269

In [249]:
data

0       [Мертвец,  Бойцовский клуб,  Королевская битва...
1       [Vanilla Sky,  I Robot,  Idiocracy,  Lock,  st...
2        [«Карты,  деньги,  два ствола»,  и много других]
3       [Rosencrantz and Guildenstern are dead,  весте...
4       ["О чем говорят мужчины",  "Мистер и миссис См...
5       [Ванильное небо,  Дом у озера,  Двухсотлетний ...
6       [Малхолланд Драйв,  Человек которого не было, ...
7       [Человек с киноаппаратом,  Modern Times,  The ...
8       [Бойцовский клуб,  Матрица,  Плезентвиль,  Тра...
9       [The Matrix,  Ocean's Eleven,  Guy Ritchie,  F...
10      [Thomas Crown Affair,  Schindler's List,  Godf...
11           [День Сурка,  Трасса 60,  Побег из Шоушенка]
12      [Devil's advocate,  Valhalla rising,  Revenger...
13      [Красота по-американски,  Стиляги,  Across the...
14      [Даун Хаус,  Шапито Шоу,  Иваново Детство,  Ка...
15      [Бойцовский клуб,  Аризонская мечта,  Сталкер,...
16      [Casablanca,  The Matrix,  North By Northwest,...
17      [Фильм

In [9]:
unique_movies = get_unique_movies(data)

In [217]:
matrix = construct_matrix(data, unique_movies)

In [218]:
svd = TruncatedSVD()
svd.fit(matrix)

TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=None, tol=0.0)

In [243]:
y = svd.inverse_transform(svd.transform(matrix.todense()[5]))

In [242]:
data[5]

['Ванильное небо',
 ' Дом у озера',
 ' Двухсотлетний человек',
 ' Побег из Шоушенка',
 ' Зеленая миля',
 ' Бой с тенью ...продолжение следует']

In [244]:
for i in range (100):
    print(unique_movies[y.argsort()[0][len(y.argsort())-i]])

 Такси 2
 " "Дворовые танцы"
 2
 3
 Побег из Шоушенка
 Бойцовский клуб
 Зеленая миля
 Достучаться до небес
 4
 Игры разума
 Матрица
 
 Гладиатор
 Начало
 Реквием по мечте
 Властелин колец
 Форест Гамп
 Эффект бабочки
 1+1
 Форрест Гамп
 Список Шиндлера
 Леон
 В погоне за счастьем
 Амели
 Жизнь прекрасна
 Криминальное чтиво
 Назад в будущее
 Вечное сияние чистого разума
 Престиж
 Молчание ягнят
 Спеши любить
 Аватар
 Пианист
 5
 Пролетая над гнездом кукушки
 Титаник
 Секрет
 Крестный отец
 Время
 Отступники
 Семь
 Заплати другому
 Дневник памяти
 Троя
 Ванильное небо
 Иван Васильевич меняет профессию
 Остров проклятых
 Сумерки
 2012
 Один дома
 Служебный роман
 Славные парни
 Трасса 60
 Форсаж
 Куда приводят мечты
 деньги
 Бриллиантовая рука
 Большой куш
 Унесённые призраками
 Иллюзионист
 Поймай меня если сможешь
 12
 Семь жизней
 Мирный воин
 6
 Апокалипсис сегодня
 Остров
 Карты
 Храброе сердце
 Социальная сеть
 99 франков
 Трансформеры
 День сурка
 В джазе только девушки
 Рок-волна
